<a href="https://colab.research.google.com/github/shinyoungin4137/AI_final_project/blob/main/final_project_finetuning__w_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial intelligence Final Project: Finetuning_BERT

Copyright (C) Computer Science & Engineering, Soongsil University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them (November 2025.)

BERT(Bidirectional Encoder Representations from Transformers) is a groundbreaking model in the NLP domain. This tutorial provides a step-by-step guide on how to fine-tune the lightweight BERT variant using Hugging Face's transformers library for text classification tasks.<br>

This is about BERT (Devlin et al., 2018).<br>
https://arxiv.org/abs/1810.04805

The code below are based on the following link. <br>
https://medium.com/@khang.pham.exxact/text-classification-with-bert-7afaacc5e49b


### Fine-tune the model
1. Design your model's prediction head
2. Finetune the model by changing the hyperparameters.
3. You will get a score based on the your (hidden) test accuracy for text classification (ranking-based).  

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that we can grade both your code and results.  


Now proceed to the code.


## Install libraries

In [ ]:
import os

In [ ]:
!python3 -m pip install pandas
!python3 -m pip install transformers

### import libraries

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import re
import torch.nn as nn

#  Preprocessing
def clean_text_bert(text):
    # 1. Remove HTML Tag
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Remove URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 3. Remove Unnecessary Spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

### Specify your GPU number if necessary

In [ ]:
%env CUDA_VISIBLE_DEVICES = 0

if torch.cuda.is_available() is True:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

env: CUDA_VISIBLE_DEVICES=0


## Preparing dataset

In [ ]:
from google.colab import drive
import os

# 1. 구글 드라이브 연결 (마운트)
drive.mount('/content/drive')

# 2. 작업 위치를 '내 프로젝트 폴더'로 이동
# (폴더 이름이 'final_project_AI'가 맞는지 꼭 확인하세요!)
project_path = '/content/drive/MyDrive/final_project_AI'

try:
    os.chdir(project_path)
    print(f"✅ 작업 위치 이동 완료! 현재 위치: {os.getcwd()}")

    # 3. 파일이 진짜 있는지 확인사살
    if os.path.exists('./data/IMDB Dataset Train.csv'):
        print("🎉 파일 찾음! 이제 로드하시면 됩니다.")
    else:
        print("⚠️ 폴더는 왔는데 파일이 안 보여요. 파일명을 확인해보세요.")
        print(f"현재 폴더 파일 목록: {os.listdir('.')}")
        if os.path.exists('data'):
             print(f"data 폴더 내부 목록: {os.listdir('data')}")

except FileNotFoundError:
    print("❌ 폴더 경로가 틀린 것 같습니다. 구글 드라이브 폴더명을 다시 확인해주세요.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 작업 위치 이동 완료! 현재 위치: /content/drive/MyDrive/final_project_AI
🎉 파일 찾음! 이제 로드하시면 됩니다.


link : https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

1. Download the dataset from attached link.
2. Move the downloaded zip file under the "data" directory and then unzip the zip file.
3. Run the following cell

In [ ]:
def load_imdb_data(data_file_path):
    if os.path.exists(data_file_path):
        df = pd.read_csv(data_file_path)
        texts = df['review'].tolist()
        labels = [1 if sentiment == "positive" else 0 for sentiment in df['sentiment'].tolist()]
        return texts, labels
    else:
        raise FileNotFoundError(f"The file '{data_file_path}' does not exist.")

data_file_path = './data/IMDB Dataset Train.csv'
texts, labels = load_imdb_data(data_file_path)

## Dataset class

In [ ]:
class CustomTextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_seq_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = clean_text_bert(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

## Classifier head for BERT( Design your model's prediction head )

In [ ]:
class CustomBERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(CustomBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)

        # Add Dropout
        self.dropout = nn.Dropout(p=0.3)
        # Add Classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        output = self.dropout(pooled_output)
        logits = self.classifier(output)

        return logits

## train and evaluation method

In [ ]:
def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in tqdm(data_loader, desc="Train"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Define Loss Function with Label Smoothing
        criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

        # Calculate Loss
        loss = criterion(outputs, labels)

        loss.backward()

        #Gredient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

def evaluate_model(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

## Hyper-parameter settings

In [ ]:
# Set up parameters
# Hint: generally, 5 ~ 10 epochs will be enough.
bert_model_name = 'bert-base-uncased'
num_classes = 2

max_seq_length = 256
batch_size = 32
num_epochs = 4
learning_rate = 2e-5

## get data utils

In [ ]:
######################## DO NOT CHANGE ########################
train_texts, val_texts, train_labels, val_labels = \
train_test_split(texts, labels, test_size=0.2, random_state=42)
######################## DO NOT CHANGE ########################

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = CustomTextClassificationDataset(train_texts, train_labels, tokenizer, max_seq_length)
val_dataset = CustomTextClassificationDataset(val_texts, val_labels, tokenizer, max_seq_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Define model, optimizer and scheduler

In [ ]:
model = CustomBERTClassifier(bert_model_name, num_classes).to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate)

total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Train model and save best model

In [ ]:
eval_acc = 0
for epoch in range(num_epochs):
    model_path = './finetuned_bert.pth'
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_model(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report = evaluate_model(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

    if eval_acc < accuracy:
        torch.save(model.state_dict(), model_path)
        print('Saved Trained Model.')
        eval_acc = accuracy

Epoch 1/4


Validation: 100%|██████████| 250/250 [02:33<00:00,  1.63it/s]


Validation Accuracy: 0.9257
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3911
           1       0.92      0.93      0.93      4089

    accuracy                           0.93      8000
   macro avg       0.93      0.93      0.93      8000
weighted avg       0.93      0.93      0.93      8000

Saved Trained Model.
Epoch 2/4


Validation: 100%|██████████| 250/250 [02:33<00:00,  1.63it/s]


Validation Accuracy: 0.9229
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      3911
           1       0.94      0.90      0.92      4089

    accuracy                           0.92      8000
   macro avg       0.92      0.92      0.92      8000
weighted avg       0.92      0.92      0.92      8000

Epoch 3/4


Validation: 100%|██████████| 250/250 [02:33<00:00,  1.63it/s]


Validation Accuracy: 0.9285
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      3911
           1       0.94      0.92      0.93      4089

    accuracy                           0.93      8000
   macro avg       0.93      0.93      0.93      8000
weighted avg       0.93      0.93      0.93      8000

Saved Trained Model.
Epoch 4/4


Validation: 100%|██████████| 250/250 [02:32<00:00,  1.64it/s]


Validation Accuracy: 0.9291
              precision    recall  f1-score   support

           0       0.93      0.92      0.93      3911
           1       0.92      0.94      0.93      4089

    accuracy                           0.93      8000
   macro avg       0.93      0.93      0.93      8000
weighted avg       0.93      0.93      0.93      8000

Saved Trained Model.
